<h1>Решение задач по второй части курса SQL<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.style.hide_index()
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


Задание 1. Выведите количество пустых значений по каждой из колонок (название колонки, кол-во пустых значений)

In [7]:
query = "SELECT "
query += ','.join(f"count(case when {column} is null then 1 end) as {column}" for column in data.columns)
query += " FROM boston"
pd.read_sql(query, con)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Задание 2. Выведите количество уникальных значений по каждой из колонок (название колонки, кол-во уникальных значений)

In [8]:
query = "SELECT "
query += ','.join(f"COUNT(DISTINCT {column}) AS {column}" for column in data.columns)
query += " FROM boston"
pd.read_sql(query, con)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,504,26,76,2,81,446,356,412,9,66,46,357,455,229


Задание 3. Выведите колонки, у которых медиана равна минимальному значению (название колонки).
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
for column in data.columns:
    query = f"""
    WITH COLUMN_RANGE AS (
        SELECT
            {column},
            ROW_NUMBER() OVER (ORDER BY {column}) rn,
            MIN({column}) OVER () MIN_{column},
            COUNT({column}) OVER () cnt
        FROM boston
    )
    SELECT {column}, MIN_{column} FROM COLUMN_RANGE
    WHERE rn = cnt/2
    """
    result = pd.read_sql(query, con)
    if result.at[0, column] == result.at[0, 'MIN_' + column]:
        print(f"У колонки {column} медиана и минимальное значение равны {result.at[0, column]}")

У колонки ZN медиана и минимальное значение равны 0.0
У колонки CHAS медиана и минимальное значение равны 0.0


- Поскольку минимальное значение и медиана совпадают, значит более половины значений колонки заполнены этим значением

Задание 4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
print("Разница между средним количеством квартир:")
top_n_group = [25, 50, 100, 200, 300]
for i in top_n_group:
    query = f"""
    SELECT ROUND(RICH - CHEAP, 2) DELTA_RM FROM (
        SELECT 
            AVG(RM) RICH
        FROM (
            SELECT
                RM
            FROM boston
            ORDER BY MEDV DESC
            limit {i}
        )
    ), (
        SELECT 
            AVG(RM) CHEAP
        FROM (
            SELECT
                RM
            FROM boston
            ORDER BY MEDV ASC
            limit {i}
        )
    )
    """
    result = pd.read_sql(query, con)
    print(f"Для {i} самых дорогих и {i} самых дешевых домов: {result.at[0, 'DELTA_RM']}")

Разница между средним количеством квартир:
Для 25 самых дорогих и 25 самых дешевых домов: 1.89
Для 50 самых дорогих и 50 самых дешевых домов: 1.73
Для 100 самых дорогих и 100 самых дешевых домов: 1.31
Для 200 самых дорогих и 200 самых дешевых домов: 0.85
Для 300 самых дорогих и 300 самых дешевых домов: 0.57


- количество квартир в среднем между дорогими и дешевыми домами есть, но небольшая - составляет 1-2 квартиры

Задание 5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [11]:
query = f"""
SELECT
    LSTAT,
    MEDV,
    dense_rank() OVER (ORDER BY LSTAT DESC) as LSTAT_rank
FROM boston
ORDER BY MEDV DESC
"""
result = pd.read_sql(query, con)
print(result)

     LSTAT  MEDV  LSTAT_rank
0     9.53  50.0         280
1     8.88  50.0         301
2     7.44  50.0         334
3     5.12  50.0         397
4     4.63  50.0         414
..     ...   ...         ...
501  23.97   7.0          41
502  29.97   6.3          12
503  26.77   5.6          26
504  30.59   5.0          11
505  22.98   5.0          50

[506 rows x 3 columns]


- в местах, где продаются дома с более высокой стоимостью, больше процент населения с высоким и средним статусом

Задание 6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
    SELECT
        CHAS,
        AVG_PRICE
    FROM (
        SELECT
            CHAS,
            ROUND(AVG(MEDV) OVER(PARTITION BY CHAS), 3) as AVG_PRICE
        FROM boston
    )
    GROUP BY CHAS, AVG_PRICE
    ORDER BY CHAS ASC
    """,
    con
)

,CHAS,AVG_PRICE
0,0.0,22.094
1,1.0,28.440


Задание 7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки). Напишите какой вывод можно сделать по этим данным.

In [13]:
for column in data.columns:
    if column == 'CHAS':
        continue
    query = f"""
    SELECT
        CHAS,
        AVG_{column}
    FROM (
        SELECT
            CHAS,
            ROUND(AVG({column}) OVER(PARTITION BY CHAS), 2) as AVG_{column}
        FROM boston
    )
    GROUP BY CHAS, AVG_{column}
    ORDER BY CHAS ASC
    """
    result = pd.read_sql(query, con)
    if result.at[1, 'AVG_' + column] > result.at[0, 'AVG_' + column]:
        print(f"У колонки {column} среднее значение выше, если дом граничит с рекой "
              f"({result.at[1, 'AVG_' + column]} > {result.at[0, 'AVG_' + column]})")

У колонки INDUS среднее значение выше, если дом граничит с рекой (12.72 > 11.02)
У колонки NOX среднее значение выше, если дом граничит с рекой (0.59 > 0.55)
У колонки RM среднее значение выше, если дом граничит с рекой (6.52 > 6.27)
У колонки AGE среднее значение выше, если дом граничит с рекой (77.5 > 67.91)
У колонки B среднее значение выше, если дом граничит с рекой (373.0 > 355.46)
У колонки MEDV среднее значение выше, если дом граничит с рекой (28.44 > 22.09)


- для колонки INDUS означает, что часть промышленных предприятий лучше строить возле реки
- для колонки NOX видимо есть взаимосвязь с количеством промышленных предприятий
- для колонки RM различие незначительное
- для колонки AGE - дома начинали строиться сначала возле реки
- для колонки B - логическая взаимосвязь непонятна
- для колонки MEDV - вид на реку положительно влияет на цену дома

Задание 8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцентиль(10, 20...100),значение INDUS, значение NOX). Напишите прослеживается ли между ними взаимосвязь

In [14]:
pd.read_sql(
    """
    SELECT PERC_DIFF, DIFF_CNT
    FROM (
        SELECT
            PERC_DIFF,
            LAST_VALUE(COUNT_DIFF) OVER(PARTITION BY PERC_DIFF ORDER BY PERC_DIFF
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) DIFF_CNT
        FROM (
            SELECT PERC_DIFF, row_number() OVER (PARTITION BY PERC_DIFF ORDER BY PERC_DIFF) COUNT_DIFF
            FROM (
                SELECT INDUS, NOX, ABS(PERC_INDUS - PERC_NOX) as PERC_DIFF
                FROM (
                    SELECT
                        INDUS, NOX, NTILE(10) OVER(ORDER BY INDUS ASC) PERC_INDUS,
                        NTILE(10) OVER(ORDER BY NOX ASC) PERC_NOX
                    FROM boston
                    ORDER BY NOX
                )
                ORDER BY PERC_DIFF DESC
            )
        )
    )
    GROUP BY PERC_DIFF, DIFF_CNT
    """,
    con
)

,PERC_DIFF,DIFF_CNT
0,0,165
1,1,188
2,2,86
3,3,35
4,4,13
5,5,9
6,6,10


абсолютная разница между перцентилями для значений INDUS и NOX в целом небольшая, что говорит о сильной положительной корреляции:
- для 165 значений перцентили совпадают
- для 188 значений перцентили отличаются на 1
- для 86 значений перцентили отличаются на 2
- для 67 значений перцентили отличаются на 3 и более